## Needed Libraries
**Pandas** is used to load text files and clean the data  
**Numpy** is used to interpolate the values at the desired location  
**Time** is used to measure execution times  
**os** is used to run terminal commands

In [153]:
import pandas as pd
import numpy as np
import time
import os

## Cleaning Process
First an example step by step is going to be shown

### Data Extraction
Here we'll use ecCodes, installation instructions can be found [here](https://confluence.ecmwf.int/display/ECC/ecCodes+installation).<br />
We'll use the command *[grib_get_data](https://confluence.ecmwf.int/display/ECC/grib_get_data)* to extract the values. <br />
The command looks like this:<br />
*grib_get_data -p dataDate,dataTime tcwv_lg.grib > tcwv_lg.txt*<br />
This will also show the date and hour of each value, and this will be saved in a text file.

In [13]:
%%bash
grib_get_data -p dataDate,dataTime RawData/tcwv_lg.grib > TCWV/LaGuardia/tcwv_lg.txt

Extracted Data initially looks like this:

In [14]:
%%bash
cat TCWV/LaGuardia/tcwv_lg.txt | head -n7

Latitude, Longitude, Value, dataDate, dataTime
  -17.750  -63.500 4.5928085327e+01 19790101 0
  -17.750  -63.250 5.0115585327e+01 19790101 0
  -18.000  -63.500 4.2574569702e+01 19790101 0
  -18.000  -63.250 4.7613632202e+01 19790101 0
Latitude, Longitude, Value, dataDate, dataTime
  -17.750  -63.500 4.8029144287e+01 19790101 600


### Data Loading
We'll use [*read_csv*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) to load the data and we'll specify the path of the file and that separators are spaces

In [17]:
df = pd.read_csv('TCWV/LaGuardia/tcwv_lg.txt', sep='\s+')

We'll use [*head()*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) to display the first lines of the dataframe <br />
Here we can see a couple of things:
 - Column indexes contain an extra comma at the end of the word
 - Column indexes are included for each point
 - Values were loaded as strings

In [20]:
df.head(7)

,"Latitude,","Longitude,","Value,","dataDate,",dataTime
0,-17.750,-63.500,4.5928085327e+01,19790101,0
1,-17.750,-63.250,5.0115585327e+01,19790101,0
2,-18.000,-63.500,4.2574569702e+01,19790101,0
3,-18.000,-63.250,4.7613632202e+01,19790101,0
4,"Latitude,","Longitude,","Value,","dataDate,",dataTime
5,-17.750,-63.500,4.8029144287e+01,19790101,600
6,-17.750,-63.250,5.2900238037e+01,19790101,600


### Changing the column indexes
We'll remove the extra comma in each column index. <br />
First we'll store the current column indexes in the variable *col_ind*<br />
And after that we'll use [*rename()*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) to change the name of each column index.

In [23]:
col_ind = list(df)
df.rename({col_ind[0]: 'Latitude', col_ind[1]: 'Longitude', col_ind[2]: 'Value', col_ind[3]: 'dataDate'}, axis=1, inplace=True)
df.head()

,Latitude,Longitude,Value,dataDate,dataTime
0,-17.750,-63.500,4.5928085327e+01,19790101,0
1,-17.750,-63.250,5.0115585327e+01,19790101,0
2,-18.000,-63.500,4.2574569702e+01,19790101,0
3,-18.000,-63.250,4.7613632202e+01,19790101,0
4,"Latitude,","Longitude,","Value,","dataDate,",dataTime


### Removing extra column indexes
We'll look in the Latitude column and we'll exclude all the rows that contain the word "*Latitude,*" using [*isin()*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)

In [25]:
df = df[~df['Latitude'].isin(['Latitude,'])]
df.head()

,Latitude,Longitude,Value,dataDate,dataTime
0,-17.750,-63.500,4.5928085327e+01,19790101,0
1,-17.750,-63.250,5.0115585327e+01,19790101,0
2,-18.000,-63.500,4.2574569702e+01,19790101,0
3,-18.000,-63.250,4.7613632202e+01,19790101,0
5,-17.750,-63.500,4.8029144287e+01,19790101,600


### Changing values to Float
We'll use [*to_numeric()*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html) to change the whole dataframe to float values

In [38]:
col_ind = list(df)
for i in col_ind:
    df[i] = pd.to_numeric(df[i])
df.head()

,Latitude,Longitude,Value,dataDate,dataTime
0,-17.75,-63.50,45.928085,19790101,0
1,-17.75,-63.25,50.115585,19790101,0
2,-18.00,-63.50,42.574570,19790101,0
3,-18.00,-63.25,47.613632,19790101,0
5,-17.75,-63.50,48.029144,19790101,600


### Hourly Data Extraction
Here we'll save the different hours desired in a list to extract each one in a different csv file 

In [152]:
hours = [0, 600, 1200, 1800]
for i in hours:
    hs = f"{int(i/100):02d}"
    path = 'TCWV/LaGuardia/' + 'tcwv_' + hs + '.csv'
    df[df['dataTime']==i].to_csv(path, index=False)

### Bilinear Interpolation for each file
Now we'll start with bilinear interpolation, we have to do a couple of functions before:
 - We have to create a list containing all the dates
 - We have to do bilinear interpolation
 - We'll save the interpolated values in a text file

### List containing all the dates
We'll use two functions, the first one to return the number of days of the month depending of the year and the second one to create the list with the dates

In [158]:
def nod(year, month):
    if month == 4 or month == 6 or month == 9 or month == 11:
        days = 30
    elif month == 2:
        days = 29 if year % 4 == 0 else 28
    else:
        days = 31
    return days
def create_dates(start_year, end_year):
    days = 0
    for i in range(start_year, end_year + 1):
        day_y = 366 if (i % 4) == 0 else 365
        days += day_y
    year = start_year
    month = 1
    day = 0
    dates = []
    for i in range(0, days):    
        if day < nod(year, month):
            day += 1
        else:
            day = 1        
            if month < 12:
                month += 1
            else:
                year += 1
                month = 1
        ms = '0' + str(month) if month<10 else str(month)
        ds = '0' + str(day) if day<10 else str(day)
        dates.append(str(year) + ms + ds)
    dates_int = [int(x) for x in dates]
    return dates, dates_int

In [162]:
dates = create_dates(1979, 2019)[1]

### Data Interpolation
We'll use three functions, the first one to return the value corresponding to a latitude and longitude in a pandas dataframe, the second one to interpolate at an specific location and the last one to save a list as a text file.

In [171]:
def findval(df, lat, lon):
    latstr = list(df)[0]
    lonstr = list(df)[1]
    df = df[df[latstr]==lat]
    df = df[df[lonstr]==lon]
    val = df['Value'].item()
    return val
def interpval(df, lat=1, lon=1):
    lat_0 = df['Latitude'].iloc[2]
    lat_1 = df['Latitude'].iloc[0]
    lon_a = df['Longitude'].iloc[0]
    lon_b = df['Longitude'].iloc[1]
    t = (lat - lat_0) / (lat_1 - lat_0)
    s = (lon - lon_a) / (lon_b - lon_a)
    X_tl = findval(df, lat_1, lon_a)
    X_tr = findval(df, lat_1, lon_b)
    X_bl = findval(df, lat_0, lon_a)
    X_br = findval(df, lat_0, lon_b)
    value = (1 - s) * (1 - t) * X_bl + (1 - s) * t * X_tl + s * (1 - t) * X_br + t * s * X_tr
    return value
def savelist(array, name='file'):
    name = name + '.txt'
    with open(name, "w") as f:
        for s in array:
            f.write(str(s) +"\n")

Now we'll interpolate for the hours previously used and for all the dates created before and finally we'll save the values in a text file.

In [176]:
start_time = time.time()
for i in hours:
    tcwv_i = []
    hs = f"{int(i/100):02d}"
    hdf = df[df['dataTime']==i]
    for j in dates:
        ddf = hdf[hdf['dataDate']==j]
        iv = interpval(ddf, -17.9074, -63.3266)
        tcwv_i.append(iv)
    path = 'TCWV/LaGuardia/' + 'itcwv_' + hs
    savelist(tcwv_i, path)
print("--- %.6s seconds ---" % (time.time() - start_time))

--- 407.74 seconds ---


Now we'll create a single function to do all of this

In [241]:
def find_iv(raw_data, output, start_year, end_year, hours, lat, lon):
    start_time = time.time()
    hours = [hours] if type(hours)!=list else hours
    name = rd[::-1][rd[::-1].find('.')+1:rd[::-1].find('/')][::-1]
    txt_name = name + '.txt'
    output_path = output + txt_name
    command = 'grib_get_data -p dataDate,dataTime ' + raw_data + ' > ' + output_path
    os.system(command)
    df = pd.read_csv(output_path, sep='\s+')
    col_ind = list(df)
    df.rename({col_ind[0]: 'Latitude', col_ind[1]: 'Longitude', col_ind[2]: 'Value', col_ind[3]: 'dataDate'}, axis=1, inplace=True)
    df = df[~df['Latitude'].isin(['Latitude,'])]
    col_ind = list(df)
    for i in col_ind:
        df[i] = pd.to_numeric(df[i])
    dates = create_dates(start_year, end_year)[1]
    for i in hours:
        tcwv_i = []
        hs = f"{int(i/100):02d}"
        hdf = df[df['dataTime']==i]
        for j in dates:
            ddf = hdf[hdf['dataDate']==j]
            iv = interpval(ddf, lat, lon)
            tcwv_i.append(iv)
        path = output + 'itcwv_' + hs
        savelist(tcwv_i, path)
    return ("--- %.6s seconds ---" % (time.time() - start_time))

### UPB

In [242]:
find_iv('RawData/tcwv_upb.grib', 'TCWV/UPB/', 1979, 2019, [0, 600, 1200, 1800], -17.399253, -66.218466)

'--- 451.33 seconds ---'

### El Alto

In [243]:
find_iv('RawData/tcwv_ea.grib', 'TCWV/ElAlto/', 1979, 2019, [0, 600, 1200, 1800], -16.51, -68.17)

'--- 437.41 seconds ---'

### San Ignacio de Moxos

In [244]:
find_iv('RawData/tcwv_sim.grib', 'TCWV/SIMoxos/', 1979, 2019, [0, 600, 1200, 1800], -14.996641, -65.640895)

'--- 419.26 seconds ---'